In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
# pip install sdmetrics

In [ ]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_selection import mutual_info_classif
from sklearn.impute import KNNImputer
# import mrmr
# from mrmr import mrmr_classif
# from sklearn_genetic import GAFeatureSelectionCV
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import cohen_kappa_score, make_scorer,confusion_matrix, matthews_corrcoef, f1_score, ConfusionMatrixDisplay
import scipy.stats as stats
import tensorflow as tf
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from collections import Counter
from itertools import chain
import sdmetrics
from sdmetrics.reports.single_table import QualityReport
import datetime
import math

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/data_tabular/fars.xlsx")

# Pre processing and EDA

In [ ]:
# Remove duplicated rows
df.drop_duplicates(inplace = True)

In [ ]:
# Define lists of continuous and categorical variables
continuous = [' AGE', ' ALCOHOL_TEST_RESULT', ' DRUG_TEST_RESULTS_(1_of_3)', ' DRUG_TEST_RESULTS_(2_of_3)', ' DRUG_TEST_RESULTS_(3_of_3)']
categorical = []
for i in df.columns.to_list():
    if i not in continuous:
        categorical.append(i)

In [ ]:
# Create NaNs for missing values for posterior imputation
for var in [' SEX' ,' SEATING_POSITION', ' RESTRAINT_SYSTEM-USE',' EJECTION',' EJECTION_PATH',' EXTRICATION',' NON_MOTORIST_LOCATION', ' TAKEN_TO_HOSPITAL', ' RELATED_FACTOR_(1)-PERSON_LEVEL', ' RELATED_FACTOR_(2)-PERSON_LEVEL', ' RELATED_FACTOR_(3)-PERSON_LEVEL', ' RACE',' HISPANIC_ORIGIN']:
    df[var] = df[var].replace('Unknown', np.nan) 
df[' AIR_BAG_AVAILABILITY/DEPLOYMENT'] = df[' AIR_BAG_AVAILABILITY/DEPLOYMENT'].replace('Unknown_(If_Airbag_Available)', np.nan)
df[' POLICE_REPORTED_ALCOHOL_INVOLVEMENT'] = df[' POLICE_REPORTED_ALCOHOL_INVOLVEMENT'].replace('Unknown_(Police_Reported)', np.nan) # ;  ----- NO QUITARRRR
df[' POLICE_REPORTED_ALCOHOL_INVOLVEMENT'] = df[' POLICE_REPORTED_ALCOHOL_INVOLVEMENT'].replace('Not_reported', np.nan) # ;  ----- NO QUITARRRR
for var in [' DRUG_TEST_TYPE', ' DRUG_TEST_TYPE_(2_of_3)' , ' DRUG_TEST_TYPE_(3_of_3)']:
    df[var] = df[var].replace('Unknown_if_Tested_for_Drugs', np.nan)
for var in [' DRUG_TEST_TYPE', ' DRUG_TEST_TYPE_(2_of_3)' , ' DRUG_TEST_TYPE_(3_of_3)']:
    df[var] = df[var].replace('Unknown_Test_Type', np.nan)
for var in [' METHOD_OF_DRUG_DETERMINATION', ' METHOD_ALCOHOL_DETERMINATION', ' POLICE-REPORTED_DRUG_INVOLVEMENT']:
    df[var] = df[var].replace('Not_Reported', np.nan)
df[' POLICE-REPORTED_DRUG_INVOLVEMENT'] = df[' POLICE-REPORTED_DRUG_INVOLVEMENT'].replace('Reported_Unknown', np.nan)
df['INJURY_SEVERITY'] = df['INJURY_SEVERITY'].replace('Unknown', 'Notknown')

In [ ]:
# Encode categories
for var in categorical:
    df[var] = df[var].astype('category')
    df[var] = df[var].cat.codes

In [ ]:
for var in categorical:
    df[var] = df[var].replace(-1, np.nan)

In [ ]:
# Eliminate columns with a high percentage of missing values
df.drop([' METHOD_OF_DRUG_DETERMINATION', ' METHOD_ALCOHOL_DETERMINATION', ' POLICE-REPORTED_DRUG_INVOLVEMENT'], axis = 1, inplace = True)
for i in [' METHOD_OF_DRUG_DETERMINATION', ' METHOD_ALCOHOL_DETERMINATION', ' POLICE-REPORTED_DRUG_INVOLVEMENT']:
    categorical.remove(i)

In [ ]:
# Deal with missing values;
missing_value_imputation = 'knn'

dff = df.copy()
if missing_value_imputation == 'not':
    dff = dff.replace(np.nan, 'Unknown')
    # CONVERTIR UNKNOWN A NUMERICO
    df = dff
if missing_value_imputation == 'eliminate':
    df = dff.dropna()
if missing_value_imputation == 'freq':
    for var in categorical:
    dff[var].fillna(df[var].mode().iloc[0], inplace = True)
    for var in continuous:
    dff[continuous].fillna(df[continuous].median().iloc[0], inplace = True)
    df = dff
if missing_value_imputation == 'knn': 
    imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean')
    imputer.fit(dff)
    Xtrans = imputer.transform(dff)
    df = pd.DataFrame(Xtrans, columns = dff.columns)

In [ ]:
binary = []
numerical = []
for var in df.columns:
    if len(pd.unique(df[var])) == 2:
        binary.append(var)
    else: 
        numerical.append(var)

In [ ]:
def Cramers_v(df, col1, col2):
    a = pd.crosstab(df[col1], df[col2], margins = False)

    X2 = stats.chi2_contingency(np.array(a), correction=False)[0]
    N = np.sum(np.array(a))
    minimum_dimension = min(np.array(a).shape)-1
    result = np.sqrt((X2/N) / minimum_dimension)

          return np.sqrt((X2/N) / minimum_dimension)

# Function to study correlation between vairables, using different coefficients for different types of variables;
def correlation_study(df, numerical, categorical):
  '''
  Generate correlation matrix considering different correlation coefficients for mixed type data. 
  Pearson: numerical-numerical
  Point-biserial: numerical-binary/categorical
  Matthews: binary-binary
  Cramers V: categorical-categorical
  '''
    corr_vals = []
    variables = categorical + numerical
    for var1 in variables:
        listvar1 = []
        for var2 in variables:
            if var2 in variables:
                if var1 in numerical and var2 in numerical:
                    corr, _ = stats.pearsonr(df[var1], df[var2])
                    listvar1.append(corr)
                    print('pearson',corr)
                if var1 in numerical and var2 in categorical:
                    pbc, pval = stats.pointbiserialr(df[var2], df[var1])
                    listvar1.append(pbc)
                    print('point bi', pbc)
                if var1 in categorical and var2 in numerical:
                    pbc, pval = stats.pointbiserialr(df[var2], df[var1])
                    listvar1.append(pbc) 
                    print('pointbi', pbc)
                if var1 in categorical and var2 in categorical:
                    listvar1.append(Cramers_v(df, var1, var2))
                    print('Cramer', Cramers_v(df, var1, var2))
                corr_vals.append(listvar1)
                # print(listvar1)
    corrdf = pd.DataFrame(corr_vals, columns = variables)
    corrdf['Var'] = variables
    corrdf.set_index('Var', inplace = True)
    corrdf.update(corrdf.abs())  
    return corrdf, corr_vals

In [ ]:
corr_df, corr_vals = correlation_study(df, continuous, categorical)

In [ ]:
sns.heatmap(corr_df)

<AxesSubplot: ylabel='Var'>


# Useful functions

## Imbalance correction

In [ ]:
# Function to evaluate the quality of synthetic data created using oversampling
def compute_quality_metrics(df_real, df_synthetic, list_categorical_vars, return_score=False):
    dict_aux = {}
    for var_name in df_real.columns.values:
        if var_name in list_categorical_vars:
            dict_aux[var_name] = {'type': 'categorical'}
        else:
            dict_aux[var_name] = {'type': 'numerical'}
    dict_metadata = {'fields': dict_aux}

    report = QualityReport()
    report.generate(df_real, df_synthetic, dict_metadata)
    df_metrics = report.get_details(property_name='Column Shapes')

    fig1 = report.get_visualization(property_name='Column Shapes')
    plt.savefig('colshapes_report.png')
    fig1.show()

    fig = report.get_visualization(property_name='Column Pair Trends')
    plt.savefig('colpairs_report.png')
    fig.show()

    if return_score:
        qscore = df_metrics.iloc[:, -1].values
        return qscore

In [ ]:
# Functions for Class-imbalance correction
def smote_oversample_all(X_train, y_train, binary,  quality_rep, return_score, sample_strategy):
    data_real = X_train
    data_real['y'] = y_train
    #sample_strategy = {1:42116, 2:42116, 3:42116, 4:42116, 5:42116, 6:42116, 7:42116}
    sm = SMOTE(sampling_strategy=sample_strategy, k_neighbors = 3, random_state=0, n_jobs = -1)
    X_res, y_res = sm.fit_resample(X_train, y_train)
    data_synthetic = X_res
    data_synthetic['y'] = y_res
    X_final = X_train.append(X_res, ignore_index = True)
    y_final = y_train.append(y_res, ignore_index = True)
    print(X_train.shape)
    print(X_final.shape)
    if quality_rep == True:
        # plt.figure(figsize=(40,40))
        quality_report = compute_quality_metrics(data_real, data_synthetic, binary, return_score=False)
        return X_final, y_final, quality_report
    else:
        return X_final, y_final

In [ ]:
def random_undersampling_all(X_train, y_train, sample_strategy):
    rus = RandomUnderSampler(sample_strategy) 
    X_rus, y_rus = rus.fit_resample(X_train, y_train)
    return X_rus, y_rus

In [ ]:
def undersample_tomek_links(X_train, y_train):
    tl = TomekLinks(n_jobs = -1)
    X_res, y_res = tl.fit_resample(X_train, y_train)
    return X_res, y_res

In [ ]:
def hybrid_imbalance(X, y):
    minn = y.value_counts()[3.0]
    # Tomek-links undersampling on majority classes
    X_pre_tomek = X.loc[y.isin([1.0, 4.0, 2.0, 5.0, 7.0])]
    y_pre_tomek = y.loc[y.isin([1.0, 4.0, 2.0, 5.0, 7.0])]
    X_tomek, y_tomek = undersample_tomek_links(X_pre_tomek, y_pre_tomek)
    # random undersampling on majority classes
    sample_strategy = {1.0:minn, 2.0:minn, 4.0:minn, 5.0:minn, 7.0:minn}
    X_u, y_u = random_undersampling_all(X_tomek, y_tomek, sample_strategy)

    # oversample minority classes
    X_no_tomek = X.loc[y.isin([0.0, 3.0, 6.0])]
    y_no_tomek = y.loc[y.isin([0.0, 3.0, 6.0])]
    X_o, y_o = smote_oversample_all(X_no_tomek, y_no_tomek, sample_strategy = {0.0: minn, 3.0:minn, 6.0:minn})

    # merge both
    X_h = X_u.append(X_o)
    y_h = y_u.append(y_o)

    return X_h, y_h

In [ ]:
# X = df.drop('INJURY_SEVERITY', axis = 1)
# y = df['INJURY_SEVERITY']

# Xo, yo, r = smote_oversample_all(X, y, [], True, False, sample_strategy = {1:42116, 2:42116, 3:42116, 4:42116, 5:42116, 6:42116, 7:42116})

## Feature scaling

In [ ]:
# Function for feature normalization;
def scale_features(x_train, x_test):
    scaler = StandardScaler()
    x_train_raw = x_train.copy()
    x_test_raw = x_test.copy()
    scaler.fit(x_train)
    x_train = pd.DataFrame(data = scaler.transform(x_train), columns = list(x_train_raw.columns))
    x_test = pd.DataFrame(data = scaler.transform(x_test), columns = list(x_train_raw.columns))

    return x_train, x_test, x_train_raw, x_test_raw

## Select classifier and hyperparametr grids

In [ ]:
# Function to retrieve untrained classifier and hyperparameter grids, to then train;
def get_hyperparams(classifier_name: str, priors, cost_sensitive, num_features, seed_value: int = 4242): 
    MAX_ITERS = 20000
    dict_param_grid_classifier = {}
    classifier = None

    if classifier_name == 'nv':
        if cost_sensitive == True:
            dict_param_grid_classifier = {'class_prior': priors}
        else:
            dict_param_grid_classifier = {'class_prior': (1/8,1/8,1/8,1/8,1/8,1/8,1/8,1/8)}
        classifier = ComplementNB()

    if classifier_name == 'svm':
        dict_param_grid_classifier = {'C': np.linspace(0.01, 10, 10),'gamma': ['auto'], 'kernel': ['rbf', 'linear', 'poly', 'sigmoid']} 
        if cost_sensitive == True:
            dict_param_grid_classifier['class_weight'] = ['balanced']
        classifier = SVC(max_iter=MAX_ITERS, random_state=seed_value)

    elif classifier_name == 'dt':
        dict_param_grid_classifier = {'max_depth': np.arange(1, 30, 1), 'min_samples_split': np.arange(2, 50, 2)}
        if cost_sensitive == True:
            dict_param_grid_classifier['class_weight'] = ['balanced']
        classifier = DecisionTreeClassifier(random_state=seed_value)

    elif classifier_name == 'rf':
        dict_param_grid_classifier = {'max_depth': np.arange(1, 30, 3), 'n_estimators': np.arange(20, 50, 5),'min_samples_split': np.arange(2, 25, 5)}
        if cost_sensitive == True:
            dict_param_grid_classifier['class_weight'] = ['balanced']
        classifier = RandomForestClassifier(n_jobs=-1, random_state=seed_value)

    elif classifier_name == 'knn':
        dict_param_grid_classifier = {'n_neighbors': np.arange(1, 20, 1)}
        classifier = KNeighborsClassifier()
  
    elif classifier_name == 'lr':
        dict_param_grid_classifier = {'penalty': ['l2', 'none']}
        if cost_sensitive == True:
            dict_param_grid_classifier['class_weight'] = ['balanced']
        classifier = LogisticRegression(multi_class='multinomial', n_jobs=-1, random_state=seed_value)

    elif classifier_name == 'mlp': # ---------------- Cambiar
        np.random.seed(seed_value)
        # random.seed(seed_value)
        tf.random.set_seed(seed_value)
        fit_kwargs = {"optimizer": "adam", "learning_rate": "adaptive", "max_iter": MAX_ITERS, "batch_size": 64, "validation_fraction": 0.15, "verbose": 0}
        dict_param_grid_classifier = {'learning_rate_init':np.linspace(0.001, 1, 10),
                                    'hidden_layer_sizes': [(num_features, 10), (num_features, 20), (num_features, 30), (num_features, 40), (num_features, 50), 
                                                           (num_features, 60), (num_features, 70), (num_features, 80), (num_features, 100)]}
        classifier = MLPClassifier(**fit_kwargs)

  

    list_dicts_params_learning_curve = []

    for key, value in dict_param_grid_classifier.items():
        dict_param_lrc = {'param_name': key, 'param_range': value}
        list_dicts_params_learning_curve.append(dict_param_lrc)

    return classifier, dict_param_grid_classifier, list_dicts_params_learning_curve

In [ ]:
cohen_scorer = make_scorer(cohen_kappa_score, greater_is_better=True)

## Feature selection techniques

In [ ]:
# Function for Genetic algorithms for feature selection;
def perform_genetic_fs(x_train, y_train, v_col_names, estimator_name, idx, priors):
    model_regressor, param_grid_regressor, list_dict_params = get_hyperparams(estimator_name, priors, cost_sensitive = True, num_features = x_train.shape[1],seed_value= idx)
    df_x_train = pd.DataFrame(x_train, columns=v_col_names)
    ga_model = GAFeatureSelectionCV(estimator=model_regressor, cv=3, scoring=cohen_scorer, population_size=30, generations=12, n_jobs=-1, verbose=True, keep_top_k=2, elitism=True,) 
    ga_model.fit(x_train, y_train)
    v_selected_features_idx = ga_model.best_features_
    v_selected_features = df_x_train.loc[:, v_selected_features_idx].columns.values
    return v_selected_features

In [ ]:
# Kruskal Wallis test for feature selection (filter method):
def perform_kruskal(X_train, y_train):
    selected = []
    pre = X_train.copy()
    pre['y'] = y_train
    for feat in pre.columns:
        if feat != 'y':  
          group0 = pre[feat].loc[pre['y'] == 0.0]
          group1 = pre[feat].loc[pre['y'] == 1.0]
          group2 = pre[feat].loc[pre['y'] == 2.0]
          group3 = pre[feat].loc[pre['y'] == 3.0]
          group4 = pre[feat].loc[pre['y'] == 4.0]
          group5 = pre[feat].loc[pre['y'] == 5.0]
          group6 = pre[feat].loc[pre['y'] == 6.0]
          group7 = pre[feat].loc[pre['y'] == 7.0]
          _,p = stats.kruskal(group0,group1, group2, group3, group4, group5, group6, group7)
          if p < 0.05:
            selected.append(feat)
    return selected

In [ ]:
def perform_mutualinfo(X,y):
    # do calculations
    mutual = mutual_info_classif(X, y, random_state = 8)
    mutual /= np.max(mutual)

    zipped_pairs = zip(list(mutual), list(X.columns))
    mutualinfo_var = [x for _, x in sorted(zipped_pairs, reverse = True)]
    mutualinfo_val = sorted(mutual, reverse = True)
    selected = []
    for idx,i in enumerate(mutualinfo_val):
        if i >= 0.1:
            selected.append(mutualinfo_var[idx])
    return selected

## Interpretability methods


In [ ]:
# This code implements the Accumulated Local effects interpretability method. The SHAP method code is inside the function: "validation_results" in the next section.
def plot_corresponding_ale_numerical_feature(df_ale_numerical, df_ale_numerical_raw, var_name, estimator_name, fes, type_over, iter, flag_save_figure=True):

    df_ale_numerical = df_ale_numerical.reset_index()
    df_ale_numerical_raw = df_ale_numerical_raw.reset_index()
    first_std = df_ale_numerical.iloc[0, 1]
    df_ale_numerical['lowerCI_95%'] = df_ale_numerical['lowerCI_95%'].fillna(first_std)
    df_ale_numerical['upperCI_95%'] = df_ale_numerical['upperCI_95%'].fillna(first_std)

    fig, ax = plt.subplots(1, 1, figsize=(5, 3.1))
    ax.plot(df_ale_numerical_raw[var_name], df_ale_numerical['eff'], ls='solid', color='blue')
    ax.fill_between(df_ale_numerical_raw[var_name], df_ale_numerical['lowerCI_95%'], df_ale_numerical['upperCI_95%'], label='95% CI', alpha=0.3, color='grey', lw=2)
    ax.set_xlabel(var_name, fontsize=12)

    plt.xticks(fontsize=11)
    plt.yticks(fontsize=11)
    plt.legend()
    plt.grid(alpha=0.5, linestyle='--')
    fig.tight_layout()

    if flag_save_figure:
        fig.savefig('{}_{}_{}_ale_{}_seed_{}.pdf'.format(estimator_name,type_over,fes,var_name,iter))
        plt.close()
    else:
        plt.show()

def plot_corresponding_ale_categorical_feature(df_ale_cat, var_name, estimator_name, fes, type_over, iter, flag_save_figure=True):

    df_ale_cat = df_ale_cat.reset_index()
    first_std = df_ale_cat.iloc[0, 1]
    df_ale_cat['lowerCI_95%'] = df_ale_cat['lowerCI_95%'].fillna(first_std)
    df_ale_cat['upperCI_95%'] = df_ale_cat['upperCI_95%'].fillna(first_std)

    fig, ax = plt.subplots(1, 1, figsize=(5, 3.1))
    ax.bar(df_ale_cat[var_name], df_ale_cat['eff'], width=0.4, color='blue', alpha=0.0)
    ax.set_ylabel('', fontsize=16)
    ax2 = ax.twinx()
    ax2.bar(df_ale_cat[var_name], df_ale_cat['size'], width=0.4, color='blue', alpha=0.3)
    ax2.set_ylabel('Size', fontsize=16)
    ax.plot([df_ale_cat.iloc[0, 0], df_ale_cat.iloc[1, 0]],[df_ale_cat.iloc[0, 1], df_ale_cat.iloc[1, 1]],'--', marker='o', color='black')

    for index, row in df_ale_cat.iterrows():
        p_inf = [row[var_name], row['lowerCI_95%']]
        p_center = [row[var_name], row['eff']]
        yerr = math.sqrt((p_center[0] - p_inf[0])**2 + (p_center[1] - p_inf[1])**2)
        ax.errorbar(row[var_name], row['eff'], yerr, fmt='o', lw=2, capsize=4, color='black')

    ax.set_xlabel(var_name, fontsize=12)
    ax.set_xticks([0, 1])
    plt.grid(alpha=0.5, linestyle='--')
    fig.tight_layout()

    if flag_save_figure:
        fig.savefig('{}_{}_{}_ale_{}_seed_{}.pdf'.format(estimator_name, type_over, fes, var_name, iter))
        plt.close()
    else:
        plt.show()

def plot_ale_features(df_x_test, v_removed_features, model_estimator, df_x_test_raw, list_vars_numerical, list_vars_categorical, estimator_name, fes, type_over, iter):

    list_vars_numerical = list(set(list_vars_numerical) - set(list(v_removed_features)))
    list_vars_categorical = list(set(list_vars_categorical) - set(list(v_removed_features)))
    print('esta es la lista', list_vars_categorical)
    for var_numerical in list_vars_numerical:
        df_ale_numerical = ale(X=df_x_test,model=model_estimator,feature=[var_numerical],plot=True,feature_type="continuous",grid_size=10,include_CI=True,C=0.95)
        df_ale_numerical_raw = ale(X=df_x_test_raw,model=model_estimator,feature=[var_numerical],plot=False,feature_type="continuous",grid_size=10,include_CI=True,C=0.95)
        plot_corresponding_ale_numerical_feature(df_ale_numerical, df_ale_numerical_raw, var_numerical,estimator_name, fes, type_over, iter)

    for var_categorical in list_vars_categorical:
        ale_discr = ale(X=df_x_test,model=model_estimator,feature=[var_categorical],feature_type="discrete",plot=True,grid_size=10,include_CI=True,C=0.95)
        plot_corresponding_ale_categorical_feature(ale_discr, var_categorical,estimator_name, fes, type_over, iter)

## Evaluation of pipelines

In [ ]:
def validation_results(X, y, classifier,imputation_type, imbalance_correction_type, fs_method, cost_sensitive):

  # Perform imbalance correction
    if imbalance_correction_type == 'cost':
        X = X
        y = y
    if imbalance_correction_type == 'hybrid':
        X, y = hybrid_imbalance(X, y)

    current_time = datetime.datetime.now()
    kf = StratifiedKFold(n_splits=5, random_state = 42, shuffle=True)
    training_scores = []
    iter = 0
    test_score_cohen = []
    test_score_f1 = []
    preds = []

    actual_labels = []

    for train_index,test_index in kf.split(X,y):
        # Generate the training and test partitions of X and Y for each iteration of CV	
        X_train, X_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
        y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]

        # Feature Scaling
        X_train_norm, X_test_norm, X_train_raw, X_test_raw = scale_features(X_train, X_test)
        if classifier in ['dt', 'rf', 'xgb']:
            X_train = X_train_raw
            X_test = X_test_raw
        else:
            X_train = X_train_norm
            X_test = X_test_norm
        actual_labels.extend(y_test)

        priors = np.array(y_train.value_counts()/len(y_train))

        # Feature Selection
        if fs_method == 'ga':
            feature_selection = perform_genetic_fs(X_train, y_train, X_train.columns, classifier, iter, priors) 
        if fs_method == 'kw':  
            feature_selection = perform_kruskal(X_train,y_train) #['CASE_STATE', ' AGE', ' SEX', ' PERSON_TYPE', ' SEATING_POSITION', ' RESTRAINT_SYSTEM-USE', ' AIR_BAG_AVAILABILITY/DEPLOYMENT', ' EJECTION', ' EJECTION_PATH', ' EXTRICATION', ' NON_MOTORIST_LOCATION',
                              #' POLICE_REPORTED_ALCOHOL_INVOLVEMENT', ' ALCOHOL_TEST_TYPE', ' ALCOHOL_TEST_RESULT', ' DRUG_TEST_TYPE',       ' DRUG_TEST_RESULTS_(1_of_3)', ' DRUG_TEST_TYPE_(2_of_3)', ' DRUG_TEST_RESULTS_(2_of_3)', 
                              # ' DRUG_TEST_TYPE_(3_of_3)', ' DRUG_TEST_RESULTS_(3_of_3)', ' HISPANIC_ORIGIN', ' TAKEN_TO_HOSPITAL', ' RELATED_FACTOR_(1)-PERSON_LEVEL', ' RELATED_FACTOR_(2)-PERSON_LEVEL', ' RELATED_FACTOR_(3)-PERSON_LEVEL', ' RACE']
        if fs_method == 'mrmr':
            feature_selection = mrmr_classif(X=X, y=y, K=10)
        if fs_method == 'mi':
            feature_selection = perform_mutualinfo(X_train,y_train) #[' RACE',  ' HISPANIC_ORIGIN',  ' TAKEN_TO_HOSPITAL',  ' ALCOHOL_TEST_TYPE', ' ALCOHOL_TEST_RESULT',  ' DRUG_TEST_RESULTS_(1_of_3)',  ' DRUG_TEST_TYPE',  ' RESTRAINT_SYSTEM-USE']
        if fs_method == 'corr':
            feature_selection = [' SEX',' ALCOHOL_TEST_TYPE',' HISPANIC_ORIGIN',' TAKEN_TO_HOSPITAL',' RACE','INJURY_SEVERITY',' ALCOHOL_TEST_RESULT'] 
        else:
            feature_selection = X_train.columns

        X_train = X_train[feature_selection] 
        X_test = X_test[feature_selection] 
        # Get Model and Parameter grid_space
        model_classifier, param_grid_classifier, list_dict_params = get_hyperparams(classifier, priors, cost_sensitive, num_features=X_train[feature_selection].shape[1], seed_value=iter)

        # Nested 5-fold CV for hyperparameter tuning and validation results
        grid_cv = GridSearchCV(estimator=model_classifier, param_grid=param_grid_classifier, scoring=cohen_scorer, cv=5, return_train_score=True, n_jobs=-1) #
        grid_cv.fit(X_train, y_train)
        best_estimator = grid_cv.best_estimator_
        training_score = grid_cv.best_score_
        training_scores.append(training_score)
        iter = iter + 1

        y_pred = best_estimator.predict(X_test)
        preds.extend(y_pred)
        test_score_cohen.append(cohen_kappa_score(y_test, y_pred))
        test_score_f1.append(f1_score(y_test, y_pred, average = 'micro'))
        cmiter=confusion_matrix(actual_labels,preds)

        # SHAP Explainability
        deita = shap.sample(X_test, 1000)
        explainer = shap.KernelExplainer(best_estimator.predict, deita)
        shap_values = explainer.shap_values(deita)
        shap_expected = explainer.expected_value
        shap_importance = (np.abs(shap_values).mean(0))
        shap.summary_plot(shap_values, deita, show=False)

        print('Test score for', classifier, 'model:', cohen_kappa_score(y_test, y_pred))

        metrics_dict = {'Date':current_time, 'Iteration':iter, 'Imbalanced_correction':imbalance_correction_type , 'Missing_imputation':imputation_type, 'Classifier':classifier, 'FS':fs_method,
                       'selected_feat':feature_selection, 'Best_params':grid_cv.best_params_, 'Training_score_iteration':grid_cv.best_score_, 
                       'Cohens_kappa_test':cohen_kappa_score(y_test, y_pred), 'F1_test':f1_score(y_test, y_pred, average = 'micro'), 'cm': cmiter}
        df_metrics = pd.read_excel("/content/drive/MyDrive/data_tabular/metrics.xlsx")
        df_metrics = df_metrics.append(metrics_dict, ignore_index=True)
        df_metrics.to_excel("/content/drive/MyDrive/data_tabular/metrics.xlsx", index=False)

    print('Mean training score for', classifier, 'model:', np.mean(training_scores))
    print('Mean score: Cohens in 5 test partitions', np.mean(test_score_cohen), np.std(test_score_cohen))
    print('Mean score: F1 in 5 test partitions', np.mean(test_score_f1), np.std(test_score_f1))

# NO IMPUTATION

In [ ]:
X = df.drop('INJURY_SEVERITY', axis = 1)
y = df['INJURY_SEVERITY']
missing_value_imputation = 'none'

## No pre-processing, cost-sensitiive, no FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

## No pre-processing, cost-sensitiive, KW FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

## No pre-processing, cost-sensitiive, MI FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

## No pre-processing, hybrid, no FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

## No pre-processing, hybrid, KW FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

## No pre-processing, hybrid, MI FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

# FREQUENCY IMPUTATION

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/data_tabular/freq_imputation.xlsx")
X = df.drop('INJURY_SEVERITY', axis = 1)
y = df['INJURY_SEVERITY']

missing_value_imputation = 'freq'

## Frequency Imputation, cost-sensitive, no FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

## Frequency Imputation, cost-sensitive, KW FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

## Frequency Imputation, cost-sensitive, MI FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

## Frequency Imputation, hybrid, no FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

## Frequency Imputation, hybrid, KW FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
#aquiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

## Frequency Imputation, hybrid, MI FS



In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

# KNN IMPUTATION

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/data_tabular/knn_imputation.xlsx")
X = df.drop('INJURY_SEVERITY', axis = 1)
y = df['INJURY_SEVERITY']

missing_value_imputation = 'knn'

## KNN Imputation, cost-sensitive, no FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='none', cost_sensitive=True)

## KNN Imputation, cost-sensitive, KW FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
# aquiiiiiiiiiiiiiiiiiiiiiii
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='kw', cost_sensitive=True)

## KNN Imputation, cost-sensitive, MI FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='cost', fs_method='mi', cost_sensitive=True)

## KNN Imputation, hybrid, no FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='none', cost_sensitive=False)

## KNN Imputation, hybrid, KW FS

In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='kw', cost_sensitive=False)

## KNN Imputation, hybrid, MI FS




In [ ]:
validation_results(X,y,'lr', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'knn', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'rf', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)

In [ ]:
validation_results(X,y,'dt', imputation_type = missing_value_imputation, imbalance_correction_type='hybrid', fs_method='mi', cost_sensitive=False)